In [54]:
import ujson
import sqlite3
from tqdm.auto import tqdm
from bs4 import BeautifulSoup
import requests as rq
from time import sleep
import random

In [55]:
def get_random_waittime():
    waittime = random.weibullvariate(1, 5)/1.5
    return waittime

In [57]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) '
                  'Chrome/89.0.4389.114 Safari/537.36',
    'Accept-Language': 'en,en-US;q=0,5',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,/;q=0.8'
}

In [45]:
con = sqlite3.connect(r'F:\temp\thesisdata\datasets.db', check_same_thread=False)

In [49]:
query_get = """select
       ART_ID,
       ARTWORK_URL
        from micro_dataset1
    where VIEWS is null and likes is null
    order by 1 asc
    ;"""

query_put = "update micro_dataset1 set LIKES = ?, VIEWS = ? where ART_ID = ?;"

cursor = con.execute(query_get)
res = cursor.fetchall()
res[:2]

[('1008695',
  'https://www.saatchiart.com/art/Painting-Look-whose-coming-to-dinner/16575/775416/view'),
 ('1020928',
  'https://www.saatchiart.com/art/Painting-Faith/20375/789992/view')]

In [58]:
for i in tqdm(res[:5]):
    url = i[1]
    art_id = i[0]
    page_html = rq.get(url, headers=headers)
    while page_html.status_code != 200:
        page_html = rq.get(url, headers=headers)
        sleep(get_random_waittime())
    soup = BeautifulSoup(page_html.content)
    j = soup.find('script', {'id': '__NEXT_DATA__'}).contents[0]
    j = ujson.loads(j)
    views = j['props']['pageProps']['initialState']['page']['data']['artwork']['views']
    likes = j['props']['pageProps']['initialState']['page']['data']['artwork']['likes']
    cursor = con.execute(query_put, (likes, views, art_id))
    con.commit()

  0%|          | 0/5 [00:00<?, ?it/s]

In [43]:
con.close()

In [24]:
def get_modules(l_):
    # Convert the JSON to a dict and extract the modules section (which contains the interestings bits)
    return ujson.loads(l_[2])['project']['project']['modules']


def get_attrs(i):
    # Extract all images along with the url of the original jpg/png
    ll = []
    for x in i:
        if x['type'] == 'image':
            ll.append({'image_id': x['id'], 'project_id': x['project_id'], 'original_image_url': x['sizes']['original']})
    return ll